# Calvin Example


## Building and training the neural network

In [6]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

Show how to load the dataset for training

In [7]:
X = torch.randn(100, 8)
Y = torch.randn(100, 1)

Define model

In [33]:


class PyTorchModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.dense_0 = nn.Linear(8, 12)
        self.dense_1 = nn.Linear(12, 1)
        self.out = nn.Linear(1, 1)

    def forward(self, x):
        x = F.relu(self.dense_0(x))
        x = F.relu(self.dense_1(x))
        x = F.sigmoid(self.out(x))
        return x

Train model on dataset

In [34]:
model = PyTorchModel()
loss_function = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.01)

dataset = TensorDataset(torch.as_tensor(X, dtype=torch.float32), torch.as_tensor(Y, dtype=torch.float32))
dataloader = DataLoader(dataset, batch_size=10)

for epoch in range(150):
    for id_batch, (x_batch, y_batch) in enumerate(dataloader):
        y_batch_pred = model(x_batch)
        loss = loss_function(y_batch_pred, y_batch.view(*y_batch_pred.shape))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if epoch % 10 == 0:
        print(f"Epoch number: {epoch} loss : {loss.item()}")

/home/fra/Hacks/imperial/OptML/_build/pip_packages/lib/python3.8/site-packages/torch/nn/functional.py:1805: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Epoch number: 0 loss : -0.23312444984912872
Epoch number: 10 loss : -14.148210525512695
Epoch number: 20 loss : -18.780498504638672
Epoch number: 30 loss : -19.313873291015625
Epoch number: 40 loss : -20.95558738708496
Epoch number: 50 loss : -23.250722885131836
Epoch number: 60 loss : -24.906482696533203
Epoch number: 70 loss : -25.78409767150879
Epoch number: 80 loss : -27.761459350585938
Epoch number: 90 loss : -30.018569946289062
Epoch number: 100 loss : -31.851566314697266
Epoch number: 110 loss : -33.668060302734375
Epoch number: 120 loss : -35.461246490478516
Epoch number: 130 loss : -37.271095275878906
Epoch number: 140 loss : -40.38231658935547


## Building the MIP formulation

Need to export to ONNX, the PyTorch ONNX exporter needs to write to a file so we generate a temporary file.

In [10]:
import torch.onnx
import tempfile
from omlt.io import write_onnx_model_with_bounds, load_onnx_neural_network_with_bounds

We also define bounds on variables

In [24]:
print(torch.min(X, 0))
lb = torch.min(X, 0).values.numpy()
ub = torch.max(X, 0).values.numpy()
input_bounds = [(float(l), float(u)) for l, u in zip(lb, ub)]
input_bounds

torch.return_types.min(
values=tensor([-2.3405, -2.1813, -2.8423, -2.6981, -2.0519, -2.8156, -2.6504, -1.8242]),
indices=tensor([47, 24,  7, 87, 11, 14,  0, 58]))


[(-2.3404769897460938, 2.1179611682891846),
 (-2.1813483238220215, 2.720587730407715),
 (-2.8422508239746094, 2.089904308319092),
 (-2.6981210708618164, 3.1534392833709717),
 (-2.0519094467163086, 2.3893017768859863),
 (-2.8155999183654785, 1.9819496870040894),
 (-2.6503889560699463, 2.9323062896728516),
 (-1.8241653442382812, 2.2772531509399414)]

PyTorch needs to trace the model execution to export it, so we defined a dummy input tensor.

In [22]:
x = torch.randn(10, 8, requires_grad=True)

Now we can write the ONNX model and load it back.

In [25]:
with tempfile.NamedTemporaryFile(suffix='.onnx', delete=False) as f:
    torch.onnx.export(
        model,
        x,
        f,
        input_names=['input'],
        output_names=['output'],
        dynamic_axes={
            'input': {0: 'batch_size'},
            'output': {0: 'batch_size'}
        }
    )
    write_onnx_model_with_bounds(f.name, None, input_bounds)
    # load back
    network_definition = load_onnx_neural_network_with_bounds(f.name)

/home/fra/Hacks/imperial/OptML/_build/pip_packages/lib/python3.8/site-packages/torch/nn/functional.py:1805: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Create Pyomo model

In [27]:
import pyomo.environ as pyo
from omlt import OmltBlock
from omlt.neuralnet import NeuralNetworkFormulation

OMLT doesn't include a formulation for sigmoid, so define it here

In [38]:
def sigmoid_activation(net_block, net, layer_block, layer):
    @layer_block.Constraint(layer.output_indexes)
    def sigmoid_activation(b, *output_index):
        # TODO: use real sigmoid
        zhat_lb, zhat_ub = b.zhat[output_index].bounds
        b.z[output_index].setlb(zhat_lb)
        b.z[output_index].setub(zhat_ub)
        return b.z[output_index] == b.zhat[output_index]


In [37]:
formulation = NeuralNetworkFormulation(
    network_definition,
    activation_constraints={'sigmoid': sigmoid_activation}
)

m = pyo.ConcreteModel()

m.nn = OmltBlock()
m.nn.build_formulation(formulation)

In [39]:
m.pprint()

1 Block Declarations
    nn : Size=1, Index=None, Active=True
        5 Set Declarations
            input_assignment_index : Size=1, Index=None, Ordered=False
                Key  : Dimen : Domain : Size : Members
                None :     1 :    Any :    8 : {0, 1, 2, 3, 4, 5, 6, 7}
            inputs_set : Size=1, Index=None, Ordered=Insertion
                Key  : Dimen : Domain : Size : Members
                None :     1 :    Any :    8 : {0, 1, 2, 3, 4, 5, 6, 7}
            layers : Size=1, Index=None, Ordered=Insertion
                Key  : Dimen : Domain : Size : Members
                None :     1 :    Any :    4 : {140615613157728, 140615614166112, 140615614167216, 140615613157440}
            output_assignment_index : Size=1, Index=None, Ordered=False
                Key  : Dimen : Domain : Size : Members
                None :     1 :    Any :    1 :    {0,}
            outputs_set : Size=1, Index=None, Ordered=Insertion
                Key  : Dimen : Domain : Size : 